In [1]:
#
# _____ ______  _______      _  _  _ _______ _______ _______  _____  __   _      _     _
#   |   |_____] |  |  |      |  |  | |_____|    |    |______ |     | | \  |       \___/ 
# __|__ |_____] |  |  |      |__|__| |     |    |    ______| |_____| |  \_|      _/   \_                                                                                       
#

In [ ]:
!pip install "tracing_auto_instrumentation[ibm]" --upgrade

In [1]:
import dotenv
import logging
import os

from enum import Enum

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing.sdk import get_lastmile_tracer

In [11]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
logger = logging.getLogger(__name__)

In [12]:
class Mode(Enum):
    GENERATE = "GENERATE"
    GENERATE_TEXT = "GENERATE_TEXT"


def init_watson_model() -> Model:
    # To display example params enter
    GenParams().get_example_values()
    generate_params = {GenParams.MAX_NEW_TOKENS: 25}

    watson_model = Model(
        model_id=ModelTypes.GRANITE_13B_CHAT_V2,
        params=generate_params,
        credentials=dict(
            api_key=os.getenv("WATSONX_API_KEY"),
            url="https://us-south.ml.cloud.ibm.com",
        ),
        space_id=os.getenv("WATSONX_SPACE_ID"),
        verify=None,
        validate=True,
    )

    return watson_model

In [32]:
from tracing_auto_instrumentation.ibm import wrap_watson

TRACE_NAME = "elementary-my-dear-watson"

tracer: LastMileTracer = get_lastmile_tracer(
    TRACE_NAME
)
watson_model: Model = init_watson_model()
wrapper = wrap_watson(watson_model, tracer)

[WARNING] 2024-06-19 16:24:22,154 __init__.py:518: Overriding of current TracerProvider is not allowed


[INFO] 2024-06-19 16:24:23,143 client.py:390: Client successfully initialized
[INFO] 2024-06-19 16:24:24,100 wml_resource.py:91: Successfully finished Get available foundation models for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-06-11&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&filters=function_text_generation&limit=200'
[DEBUG] 2024-06-19 16:24:24,100 wml_resource.py:96: Response(GET https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-06-11&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&filters=function_text_generation&limit=200): {"total_count":20,"limit":200,"first":{"href":"https://us-south.ml.cloud.ibm.com/ml/v1/foundation_model_specs?version=2024-06-11&space_id=c6dd344f-1aab-4b5e-8edb-2a21fa1e0de0&filters=function_text_generation&limit=200"},"resources":[{"model_id":"bigscience/mt0-xxl","label":"mt0-xxl-13b","provider":"BigScience","source":"Hugging Face","functions":[{"id":"text_generation"}],"short_description":"A

In [18]:
def run_generation(mode: Mode, prompt: str) -> None:
    logger.info("IBM WatsonX Generation Script Starting...")

    # n.b. required for multiple api keys, make sure `.env` is in your local path
    dotenv.load_dotenv()
    
    response: str = ""
    if Mode.GENERATE == mode:
        logger.info(f"running with mode: {mode}")
        response = wrapper.generate(prompt)
    elif Mode.GENERATE_TEXT == mode:
        logger.info(f"running with mode: {mode}")
        response = wrapper.generate_text(prompt)
    else:
        logger.error(f"unsupported mode: {mode}")
    logger.info(f"response: {response}")

In [34]:
response = run_generation(
    mode=Mode.GENERATE,
    prompt="Can you please tell me an amusing anecdote about Thomas Edison?",
)

response

[INFO] 2024-06-19 16:29:21,647 1619447352.py:2: IBM WatsonX Generation Script Starting...
[INFO] 2024-06-19 16:29:21,649 1619447352.py:9: running with mode: Mode.GENERATE
[INFO] 2024-06-19 16:29:22,598 wml_resource.py:91: Successfully finished generate for url: 'https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-06-11'
[DEBUG] 2024-06-19 16:29:22,598 wml_resource.py:96: Response(POST https://us-south.ml.cloud.ibm.com/ml/v1/text/generation?version=2024-06-11): {"model_id":"ibm/granite-13b-chat-v2","model_version":"2.1.0","created_at":"2024-06-19T20:29:22.565Z","results":[{"generated_text":"\n\nCertainly! One of Edison's most famous inventions was the phonograph, the world's first practical device for recording and","generated_token_count":25,"input_token_count":13,"stop_reason":"max_tokens"}]}
[INFO] 2024-06-19 16:29:22,816 1619447352.py:16: response: {'model_id': 'ibm/granite-13b-chat-v2', 'model_version': '2.1.0', 'created_at': '2024-06-19T20:29:22.565Z', 'results': 